In [1]:
orb_fil = '/data/data_geodyn/results/icesat2/msis2/msis2_acceloff_developer_files/ORBITS/icesat2_2018314_54hr.msis2_orb2'

In [2]:
# header_labels = [
#     '-9 ,000 ,000 ,000.0D0',
#     'Number of alphanumeric data buffers to follow .(NA)',
#     'Number of card images in the GEODYN II input control deck (NC).',
#     'Arc Number.',
#     'Global Iteration Number',
#     'Inner Iteration Number',
#     'Number of satellites on this file (NSATS =1 or number of sats. in Set if Master and Slaves to be concurrently output .) This quantity has an upper limit of 50.',
#     'Actual number of words per satellite per time point (NWDSAT <= 39)',
#     'Number of words of data per time point (NWDATA=NSATS*NWDSAT )','
#     'Number of time points per Data Buffer (NTIMBF ).',
#     'Trajectory Start Date & Time in form YYMMDDHHMMSS .0D0 UTC 
#     'Fractional seconds of Start Time. UTC',
#     'Trajectory Stop Date & Time in form YYMMDDHHMMSS .0D0 UTC ',
#     'Fractional seconds of Stop Time. UTC',',
#     'Trajectory Start Date & Time in Modified Julian Day Seconds (MJDS=(JD -2430000.5 D0 )*86400+ ISEC) ET',
#     'Fractional seconds of Start Time',
#     'Trajectory Stop Date & Time in Modified Julian Day Seconds (MJDS=(JD -2430000.5 D0 )*86400+ ISEC). ET',
#     'Fractional seconds of Stop Time. ET',
#     'Nominal interval between trajectory times in seconds. ET ',
#     'Nominal number of trajectory times.',
#     'zero',
#     'Output S/C ephemeris reference system index (0 = TOD, 1 = TOR, 2 = Mean of J2000)',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'Speed of Light.',
#     'GM for Earth.',
#     'Semi -major axis of Earth reference ellipsoid.',
#     'Equatorial Flattening of Earth reference ellipsoid.',
#     'Gravitational Potential Checksum.',
#     'Maximum Degree of Gravitational Expansion.',
#     'Maximum Order of Gravitational Expansion.',    ### SKIP from 108 -200
# ]
    










In [3]:
def read_binary_ORBFIL(orb_fil):
    

    from scipy.io import FortranFile
    import numpy as np
    import pandas as pd
    from collections import namedtuple
    import time
    # Fortran calls
    import subprocess
    import os
    
    
    
    f = FortranFile(orb_fil, 'r')

    #### -----------------------------------------------------
    #### ------------------- HEADER RECORD -------------------
    #### -----------------------------------------------------
    ### Read the first record, this is the header buffer
    a = f.read_record(float)  # read the record with the required datatype


    #### Glean important variables
    NA     = int(a[2-1]) # Number of alphanumeric buffers to follow the header
    NC     = int(a[3-1]) # Number of card images in the GEODYN II input control deck
    NSATS  = int(a[7-1])  # Number of satellites on this file:  
    NWDSAT = int(a[8-1])  # Actual number of words per satellite per time point (NWDSAT <= 39).
    NWDATA = int(a[9-1])   #NSATS*NWDSAT
    NTIMBF = int(a[10-1]) # Number of time points per Data Buffer

    header= {}
    header['Number of alphanumeric data buffers to follow (NA)']             = a[2-1]
    header['Number of card images in the GEODYN II input control deck (NC)'] = a[3-1]
    header['Arc Number.']                                                    = a[4-1]
    header['Global Iteration Number']                                        = a[5-1]
    header['Inner Iteration Number']                                         = a[6-1]
    header['Number of satellites on this file']                              = a[7-1]  # upper lmit of 50
    header['Actual number of words per satellite per time point']            = a[8-1]
    header['Number of words of data per time point (NWDATA=NSATS*NWDSAT)']   = a[9-1]
    header['Number of time points per Data Buffer (NTIMBF)']                 = a[10-1]
    header['Trajectory Start Date & Time in form YYMMDDHHMMSS .0D0 UTC']     = a[11-1]
    header['Fractional seconds of Start Time. UTC']                          = a[12-1]
    header['Trajectory Stop Date & Time in form YYMMDDHHMMSS .0D0 UTC']      = a[13-1]
    header['Fractional seconds of Stop Time. UTC']                           = a[14-1]
    header['Trajectory Start Date & Time in MJDS']                           = a[15-1] # (MJDS=(JD -2430000.5 D0 )*86400+ ISEC) ET
    header['Fractional seconds of Start Time']                               = a[16-1]
    header['Trajectory Stop Date & Time in MJDS']                            = a[17-1] # (MJDS=(JD -2430000.5 D0 )*86400+ ISEC) ET
    header['Fractional seconds of Stop Time. ET']                            = a[18-1]
    header['Nominal interval between trajectory times in seconds. ET ']      = a[19-1]
    header['Nominal number of trajectory times.']                            = a[20-1]
    header['Output S/C ephem ref sys(0 = TOD, 1 = TOR, 2 =  J2000)']         = a[22-1]
    # 
    header['Speed of Light.']                                     = a[101-1]
    header['GM for Earth.']                                       = a[102-1]
    header['Semi -major axis of Earth reference ellipsoid.']      = a[103-1]
    header['Equatorial Flattening of Earth reference ellipsoid.'] = a[104-1]
    header['Gravitational Potential Checksum.']                   = a[105-1]
    header['Maximum Degree of Gravitational Expansion.']          = a[106-1]
    header['Maximum Order of Gravitational Expansion.']           = a[107-1]   ### SKIP from 108 -200
    #
    #### PRESENCE ON FILE INDICATORS
    ## right ascension of Greenwich 
    header['Presence of right ascension of Greenwich for each time point in each Buffer'] = a[201-1]   
    ## Inertial State Vector
    header['Presence per Sat. of inertial X coordinate for each time point']    = a[202-1]   
    header['Presence per Sat. of inertial Y coordinate for each time point']    = a[203-1]   
    header['Presence per Sat. of inertial Z coordinate for each time point']    = a[204-1]   
    header['Presence per Sat. of inertial Xdot coordinate for each time point'] = a[205-1]   
    header['Presence per Sat. of inertial Ydot coordinate for each time point'] = a[206-1]   
    header['Presence per Sat. of inertial Zdot coordinate for each time point'] = a[207-1] 
    # 
    header['Presence per Sat. of geodetic latitude for each time point'] = a[208-1]   
    header['Presence per Sat. of east longitude for each time point']    = a[209-1]   
    # 
    header['Presence per Sat. of ECF X coordinate for each time point']  = a[210-1]   
    header['Presence per Sat. of ECF Y coordinate for each time point']  = a[211-1]   
    header['Presence per Sat. of ECF Z coordinate for each time point']  = a[212-1]   
    header['Presence per Sat. of ECF Xdot for each time point']         = a[213-1]   
    header['Presence per Sat. of ECF Ydot for each time point']         = a[214-1]   
    header['Presence per Sat. of ECF Zdot for each time point']         = a[215-1]   
    header['Presence per Sat. of polar motion X for each time point']   = a[216-1]   
    header['Presence per Sat. of polar motion Y for each time point']   = a[217-1]   
    header['Presence per Sat. of beta prime angle for each time point'] = a[218-1]   
    header['Presence per Sat. of yaw angle for each time point']        = a[219-1]   
    header['Presence per Sat. of orbit angle for each time point']      = a[220-1]   

    ##### Satellite ID ’s for all Satellites on File.
    ###       Trajectory data is ordered based upon order of these Satellite ID ’s.'
    for i in range(int(NSATS)):
        ii = i + 1
        index_sats = 300 + (ii)
        header['Satellite '+str(ii)+' ID'] = a[index_sats-1]  


    #### ----------------------------------------------------
    #### --------------- ALPHANUMERIC RECORDS ---------------
    #### ----------------------------------------------------
    #### We don't care about the Alphanumeric buffers so skip over them.
    for i in range(int(NA)):
        a = f.read_record(float)


    #### -----------------------------------------------------
    #### -------------- DATA + SENTINEL RECORDS --------------
    #### -----------------------------------------------------
    ### Read the Data records in a while loop.  
    ### When we hit the end_data_val, we have reached the
    ###    sentinel record and we can exit the while loop 
    ###    to read in the sentinel buffer.


    end_data_val           = 9000000000
    end_datarecord         = False
    data_dict_times        = {}
    data_dict_RA_greenwich = {}
    data_dict_sat_packets  = {}

    count_while = 0

    data_dict_sat_packets['MJDSEC ET']                       =[]
    data_dict_sat_packets['Satellite Inertial X coordinate'] =[]
    data_dict_sat_packets['Satellite Inertial Y coordinate'] =[]
    data_dict_sat_packets['Satellite Inertial Z coordinate'] =[]
    data_dict_sat_packets['Satellite Inertial X velocity']   =[]
    data_dict_sat_packets['Satellite Inertial Y velocity']   =[]
    data_dict_sat_packets['Satellite Inertial Z velocity']   =[]
    data_dict_sat_packets['Satellite Geodetic Latitude']     =[]
    data_dict_sat_packets['Satellite East Longitude']        =[]
    data_dict_sat_packets['Satellite Height']                =[]
    data_dict_sat_packets['Satellite ECF X coordinate']      =[]
    data_dict_sat_packets['Satellite ECF Y coordinate']      =[]
    data_dict_sat_packets['Satellite ECF Z coordinate']      =[]
    data_dict_sat_packets['Satellite ECF X velocity']        =[]
    data_dict_sat_packets['Satellite ECF Y velocity']        =[]
    data_dict_sat_packets['Satellite ECF Z velocity']        =[]
    data_dict_sat_packets['Polar Motion X']                  =[]
    data_dict_sat_packets['Polar Motion Y']                  =[]
    data_dict_sat_packets['Beta prime angle']                =[]
    data_dict_sat_packets['Yaw angle']                       =[]
    data_dict_sat_packets['Orbit Angle']                     =[]
    data_dict_sat_packets['Q(1)']                            =[]
    data_dict_sat_packets['Q(2)']                            =[]
    data_dict_sat_packets['Q(3)']                            =[]
    data_dict_sat_packets['Q(4)']                            =[]

    while end_datarecord == False:

        ### Read in each data buffer
        a = f.read_record(float)

        if not end_data_val in a:
            count_while+=1
            NTB    = int(a[5-1])  # Number of trajectory times in this Data Buffer (NTB <= NTIMBF ).
            MJDSBF = a[4-1]

            #### Trajectory Times in elapsed ET seconds from MJDSBF
            counter = 0
            for itime in np.arange( (6)   ,   ((NTB+5)  +1)  ):
                index_times = int(itime)
                data_dict_times[counter] = MJDSBF + a[index_times-1] 
                counter+=1


            #### Right Ascension of Greenwich Values (radians) for each time in Buffer.
            counter = 0
            for i in np.arange((NTIMBF+6) ,((NTIMBF+5 + NTB)+1)):
                counter+=1
                index = int(i)
                data_dict_RA_greenwich['Right Ascension of Greenwich Values '+ str(counter)] = a[index-1] 


            ##### Satellite Data Packets
            #####    first satellite 
            #####    first time point 
            counter = 0        
            first_sat_first_time = ((NSATS +1)* NTIMBF +6) + (NSATS -1)* NWDSAT #2* NTIMBF +6
            last_sat_last_time   = ((NSATS +1)* NTIMBF +5) + NSATS*NWDSAT*NTB #(((NSATS+1)* NTIMBF+5)+(NSATS*NWDSAT))

    #         print('first_sat_first_time', first_sat_first_time)
    #         print('last_sat_last_time  ', last_sat_last_time)




            for i in np.arange(first_sat_first_time, last_sat_last_time  , 24):
                index = int(i)


                data_dict_sat_packets['MJDSEC ET'].append(data_dict_times[counter])
                data_dict_sat_packets['Satellite Inertial X coordinate'].append(a[(index +1) - 2])
                data_dict_sat_packets['Satellite Inertial Y coordinate'].append(a[(index +2) - 2])
                data_dict_sat_packets['Satellite Inertial Z coordinate'].append(a[(index +3) - 2])
                data_dict_sat_packets['Satellite Inertial X velocity'].append(a[(index +4) - 2])
                data_dict_sat_packets['Satellite Inertial Y velocity'].append(a[(index +5) - 2])
                data_dict_sat_packets['Satellite Inertial Z velocity'].append(a[(index +6) - 2])
                data_dict_sat_packets['Satellite Geodetic Latitude'].append(a[(index +7) - 2])
                data_dict_sat_packets['Satellite East Longitude'].append(a[(index +8) - 2])
                data_dict_sat_packets['Satellite Height'].append(a[(index +9) - 2])
                data_dict_sat_packets['Satellite ECF X coordinate'].append(a[(index +10) - 2])
                data_dict_sat_packets['Satellite ECF Y coordinate'].append(a[(index +11) - 2])
                data_dict_sat_packets['Satellite ECF Z coordinate'].append(a[(index +12) - 2])
                data_dict_sat_packets['Satellite ECF X velocity'].append(a[(index +13) - 2])
                data_dict_sat_packets['Satellite ECF Y velocity'].append(a[(index +14) - 2])
                data_dict_sat_packets['Satellite ECF Z velocity'].append(a[(index +15) - 2])
                data_dict_sat_packets['Polar Motion X'].append(a[(index +16) - 2])
                data_dict_sat_packets['Polar Motion Y'].append(a[(index +17) - 2])
                data_dict_sat_packets['Beta prime angle'].append(a[(index +18) - 2])
                data_dict_sat_packets['Yaw angle'].append(a[(index +19) - 2])
                data_dict_sat_packets['Orbit Angle'].append(a[(index +20) - 2])
                data_dict_sat_packets['Q(1)'].append(a[(index +21) - 2])
                data_dict_sat_packets['Q(2)'].append(a[(index +22) - 2])
                data_dict_sat_packets['Q(3)'].append(a[(index +23) - 2])
                data_dict_sat_packets['Q(4)'].append(a[(index +24) - 2])
                counter+=1


    #         print('counter',counter)    

        else:
            ####  If the the first index has +9000000000 we are at the sentinel record 
            #     which denotes the end of the data section.
            print('----- End of file')
            print('sentinel buffer indicator                       ',a[1-1])
            print('Count of the number of Data Buffers. GEODYN     ',a[2-1])
            print('GEODYN II Interface File creation date and time.',a[3-1])
            print('GEODYN II -S version used.                      ',a[4-1])
            print('GEODYN II -E version used.                      ',a[5-1])
            print('spare                                           ',a[6-1])
            print('spare                                           ',a[7-1])
            end_datarecord = True
            f.close()  #### be sure to close the file


    data_record_df = pd.DataFrame.from_dict(data_dict_sat_packets, orient='columns')

    
    orbfil_dict = {}
    orbfil_dict['header'] = header
    orbfil_dict['data_record'] = data_record_df

    return(orbfil_dict)


        

In [4]:
# data_dict_sat_packets

In [5]:
#             # in this loop, the counter is the satellite # in order
#             data_dict_sat_packets[data_dict_times[counter]] = {}
#             data_dict_sat_packets[data_dict_times[counter]]['Satellite Inertial X coordinate'] = a[(index +1) - 2]
#             data_dict_sat_packets[data_dict_times[counter]]['Satellite Inertial Y coordinate'] = a[(index +2) - 2]
#             data_dict_sat_packets[data_dict_times[counter]]['Satellite Inertial Z coordinate'] = a[(index +3) - 2]
#             data_dict_sat_packets[data_dict_times[counter]]['Satellite Inertial X velocity']   = a[(index +4) - 2]
#             data_dict_sat_packets[data_dict_times[counter]]['Satellite Inertial Y velocity']   = a[(index +5) - 2]
#             data_dict_sat_packets[data_dict_times[counter]]['Satellite Inertial Z velocity']   = a[(index +6) - 2]
#             data_dict_sat_packets[data_dict_times[counter]]['Satellite Geodetic Latitude']     = a[(index +7) - 2]
#             data_dict_sat_packets[data_dict_times[counter]]['Satellite East Longitude']        = a[(index +8) - 2]
#             data_dict_sat_packets[data_dict_times[counter]]['Satellite Height']                = a[(index +9) - 2]
#             data_dict_sat_packets[data_dict_times[counter]]['Satellite ECF X coordinate']      = a[(index +10) - 2]
#             data_dict_sat_packets[data_dict_times[counter]]['Satellite ECF Y coordinate']      = a[(index +11) - 2]
#             data_dict_sat_packets[data_dict_times[counter]]['Satellite ECF Z coordinate']      = a[(index +12) - 2]
#             data_dict_sat_packets[data_dict_times[counter]]['Satellite ECF X velocity']        = a[(index +13) - 2]
#             data_dict_sat_packets[data_dict_times[counter]]['Satellite ECF Y velocity']        = a[(index +14) - 2]
#             data_dict_sat_packets[data_dict_times[counter]]['Satellite ECF Z velocity']        = a[(index +15) - 2]
#             data_dict_sat_packets[data_dict_times[counter]]['Polar Motion X']                  = a[(index +16) - 2]
#             data_dict_sat_packets[data_dict_times[counter]]['Polar Motion Y']                  = a[(index +17) - 2]
#             data_dict_sat_packets[data_dict_times[counter]]['Beta prime angle']                = a[(index +18) - 2]
#             data_dict_sat_packets[data_dict_times[counter]]['Yaw angle']                       = a[(index +19) - 2]
#             data_dict_sat_packets[data_dict_times[counter]]['Orbit Angle']                     = a[(index +20) - 2]
#             data_dict_sat_packets[data_dict_times[counter]]['Q(1)']                            = a[(index +21) - 2]
#             data_dict_sat_packets[data_dict_times[counter]]['Q(2)']                            = a[(index +22) - 2]
#             data_dict_sat_packets[data_dict_times[counter]]['Q(3)']                            = a[(index +23) - 2]
#             data_dict_sat_packets[data_dict_times[counter]]['Q(4)']                            = a[(index +24) - 2]


# data_record = pd.DataFrame({ 'MJDSEC ET'                                                   :  [],
#                              #'Data Buffer Start Date & Time in form YYMMDDHHMMSS .0D0 UTC':  [],
#                              #'Fractional seconds of Start Time UTC '                      :  [],
#                              #'Data Buffer Start Date & Time in MJDSecs ET'                :  [],
#                              #'NTB'                             :  [], # Number of trajectory times in this Data Buffer
#                              'Satellite Inertial X coordinate' :  [],
#                              'Satellite Inertial Y coordinate' :  [],
#                              'Satellite Inertial Z coordinate' :  [],
#                              'Satellite Inertial X velocity'   :  [],
#                              'Satellite Inertial Y velocity'   :  [],
#                              'Satellite Inertial Z velocity'   :  [],
#                              'Satellite Geodetic Latitude'     :  [],
#                              'Satellite East Longitude'        :  [],
#                              'Satellite Height'                :  [],
#                              'Satellite ECF X coordinate'      :  [],
#                              'Satellite ECF Y coordinate'      :  [],
#                              'Satellite ECF Z coordinate'      :  [],
#                              'Satellite ECF X velocity'        :  [],
#                              'Satellite ECF Y velocity'        :  [],
#                              'Satellite ECF Z velocity'        :  [],
#                              'Polar Motion X'                  :  [],
#                              'Polar Motion Y'                  :  [],
#                              'Beta prime angle'                :  [],
#                              'Yaw angle'                       :  [],
#                              'Orbit Angle'                     :  [],
#                              'Q(1)'                            :  [],    #  Quaternions describing
#                              'Q(2)'                            :  [],    #  the total rotation from
#                              'Q(3)'                            :  [],   # ICRS(J200) to ITRF for
#                              'Q(4)'                            :  [],    #  this time             
#                             })


# 1     Satellite Inertial X coordinate   (meters) 
# 2     Satellite Inertial X coordinate   (meters) 
# 3     Satellite Inertial Z coordinate   (meters) 
# 4     Satellite Inertial X velocity     (meters/second)
# 5     Satellite Inertial Y velocity     (meters/second)
# 6     Satellite Inertial Z velocity     (meters/second)
# 7     Satellite Geodetic Latitude       (degrees)
# 8     Satellite East Longitude          (degrees)
# 9     Satellite Height                  (meters)
# 10    Satellite ECF X coordinate        (meters)
# 11    Satellite ECF Y coordinate        (meters)
# 12    Satellite ECF Z coordinate        (meters)
# 13    Satellite ECF X velocity          (meters/second)
# 14    Satellite ECF Y velocity          (meters/second)
# 15    Satellite ECF Z velocity          (meters/second)
# 16    Polar Motion X                    (milliarcsec)
# 17    Polar Motion Y                    (milliarcsec)
# 18    Beta prime angle                  (degrees)
# 19    Yaw angle                         (degrees)
# 20    Orbit Angle                       (degrees)
# 21    Q(1) -|  Quaternions describing 
# 22    Q(2)  |  the total rotation from 
# 23    Q(3)  |  ICRS(J200) to ITRF for
# 24    Q(4) _|  this time

In [6]:
# get_last_packet

In [7]:
#     '-9 ,000 ,000 ,000.0D0',
#     'Number of alphanumeric data buffers to follow .(NA)',
#     'Number of card images in the GEODYN II input control deck (NC).',
#     'Arc Number.',
#     'Global Iteration Number',
#     'Inner Iteration Number',
#     'Number of satellites on this file (NSATS =1 or number of sats. in Set if Master and Slaves to be concurrently output .) This quantity has an upper limit of 50.',
#     'Actual number of words per satellite per time point (NWDSAT <= 39)',
#     'Number of words of data per time point (NWDATA=NSATS*NWDSAT )','
#     'Number of time points per Data Buffer (NTIMBF ).',
#     'Trajectory Start Date & Time in form YYMMDDHHMMSS .0D0 UTC 
#     'Fractional seconds of Start Time. UTC',
#     'Trajectory Stop Date & Time in form YYMMDDHHMMSS .0D0 UTC ',
#     'Fractional seconds of Stop Time. UTC',',
#     'Trajectory Start Date & Time in Modified Julian Day Seconds (MJDS=(JD -2430000.5 D0 )*86400+ ISEC) ET',
#     'Fractional seconds of Start Time',
#     'Trajectory Stop Date & Time in Modified Julian Day Seconds (MJDS=(JD -2430000.5 D0 )*86400+ ISEC). ET',
#     'Fractional seconds of Stop Time. ET',
#     'Nominal interval between trajectory times in seconds. ET ',
#     'Nominal number of trajectory times.',
#     'zero',
#     'Output S/C ephemeris reference system index (0 = TOD, 1 = TOR, 2 = Mean of J2000)',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'spare',
#     'Speed of Light.',
#     'GM for Earth.',
#     'Semi -major axis of Earth reference ellipsoid.',
#     'Equatorial Flattening of Earth reference ellipsoid.',
#     'Gravitational Potential Checksum.',
#     'Maximum Degree of Gravitational Expansion.',
#     'Maximum Order of Gravitational Expansion.',    ### SKIP from 108 -200
    
    










In [8]:
# print('Trajectory Start Date & Time in Modified Julian Day',a[14])

# print('Nominal interval between trajectory times in seconds. ET',a[18])
# print('Nominal number of trajectory times.',a[19])
# print('zero', a[20])
# print('Output S/C ephemeris reference system index (0 = TOD, 1 = TOR, 2 = Mean of J2000)', a[21])
# print('spare', a[22])


# print('speed of light', a[100])

In [9]:
# f = FortranFile(__rvg_filename, 'r')

# end_data_val = 9000000000 #-999.0
# # end_datarecord = False
# # counter = 0



# ####   Loop through the binary file and save out each full record. 
# #      when we encounter the -999.0 delimeter at the start of the sentnial,
# #      we have reached the end of the header record.
# #
# #      The data is saved into a DataFrame for "simplicity"


# # while end_datarecord == False:

# a = f.read_record(float)  # read the record with the required datatype





In [10]:

# if end_data_val in a:
#     ####  If the the first index has -999.0 we are at the sentinel record 
#     #     which denotes the end of the data section.
#     print(self.tabtab, '----- End of file')

#     rvg_data['sentinel'] = dict(zip(sentinel_titles, a))    
#     end_datarecord = True
#     counter += 1
#     f.close()  # be sure to close the file
#     break  
# else:
#     if counter == 0:
#         #### If the counter is 0 we are on the header record.
#         #    this is simply because it is the first record. bottabing bottaboom
#         rvg_data['header'] = dict(zip(header_titles, a))    
#     else:
#         #### Everything in the file that isn't header or senitinel is data
#         rvg_data['data'].loc[counter-1] = dict(zip(data_titles,a) ) 
#     counter += 1
